In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
import itertools
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, matthews_corrcoef, accuracy_score

# Load datasets

In [3]:
# Parse the input FASTA file
train_sequences = SeqIO.parse("Data/Train.fasta","fasta")
test_sequences = SeqIO.parse("Data/Test.fasta","fasta")

# Function to generate k-mer from a protein sequence

In [4]:
def generate_unique_kmers(k=3):
    # Define the amino acid alphabet plus dummy character 'X'
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                   'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'X']

    # Generate all possible k-mers (combinations of amino acids)
    kmers = [''.join(p) for p in itertools.product(amino_acids, repeat=k)]

    # Return the list of unique k-mers plus '000' token
    return kmers + ['000']  # Add '000' token for padding


In [5]:
def generate_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return kmers

In [ ]:
generate_kmers('ABCDEFG')

In [7]:
def process_sequence(sequence, k=3, max_kmers=1000):
    kmers = generate_kmers(sequence, k)
    if len(kmers) > max_kmers:
        kmers = kmers[:max_kmers]  # Truncate if too long
    elif len(kmers) < max_kmers:
        kmers += ['000'] * (max_kmers - len(kmers))  # Pad if too short
    return kmers

Here, we truncate or pad sequences to a fixed max_kmers length (`1000` in this case), so they can be processed uniformly by the CNN.

# One-hot encoding for k-mer

We need to map each k-mer to a unique index and then perform one-hot encoding.

In [8]:
def one_hot_encode_kmers(sequence, k=3, max_kmers=1000):
    # Generate all unique k-mers plus '000' for padding
    unique_kmers = generate_unique_kmers(k=k)

    # Process the input sequence to generate k-mers, truncated/padded to max_kmers
    kmer_seq = process_sequence(sequence, k=k, max_kmers=max_kmers)

    # Initialize the OneHotEncoder using the unique k-mers as categories
    encoder = OneHotEncoder(categories=[unique_kmers],
                            sparse_output=False,
                            handle_unknown='ignore')

    # One-hot encode the k-mers (reshape to 2D array for OneHotEncoder)
    one_hot_kmers = encoder.fit_transform(np.array(kmer_seq).reshape(-1, 1))

    return one_hot_kmers

# Function to generate k_mer

In [9]:
# Function to generate k-mers in batches from a FASTA file
# Function to generate k-mers in batches from a FASTA file, now extracting labels from the file itself
def kmer_generator_for_file(fasta_file, batch_size, k=3, max_kmers=1000):
    while True:
        sequences = []
        labels = []

        # Parse the sequences from the FASTA file
        for record in SeqIO.parse(fasta_file, "fasta"):

            #Extract sequence
            seq = str(record.seq)

            # Extract label
            label = int(record.id.split('_')[-1])  # 0 or 1

            # One-hot encode the k-mers (assuming one_hot_encode_kmers function)
            encoded_kmers = one_hot_encode_kmers(seq)

            sequences.append(encoded_kmers)
            labels.append(label)

            # If batch size is met, yield the batch
            if len(sequences) == batch_size:
                yield np.array(sequences), np.array(labels)
                sequences = []
                labels = []


# Building CNN Model

In [10]:
# Define the CNN model
def build_cnn_model(input_length, num_kmer_categories):
    model = Sequential([
        # 1st Convolutional layer
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(input_length, num_kmer_categories)),

        # Pooling layer to reduce dimensionality
        MaxPooling1D(pool_size=2),

        # 2nd Convolutional layer
        Conv1D(filters=128, kernel_size=3, activation='relu'),

        # Pooling layer
        MaxPooling1D(pool_size=2),

        # Global average pooling to reduce the data to a fixed size
        GlobalAveragePooling1D(),

        # Fully connected layer
        Dense(64, activation='relu'),

        # Output layer (for binary classification, sigmoid activation)
        Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
# Example input parameters
max_kmers = 1000  # Number of k-mers in each sequence (same as in the one-hot encoding)
num_kmer_categories = len(generate_unique_kmers(k=3))  # Total number of possible k-mers (including 'PAD')

# Build the CNN model
model = build_cnn_model(input_length=max_kmers, num_kmer_categories=num_kmer_categories)

# Print the model summary to check the architecture
model.summary()


# Train CNN Model

In [ ]:
batch_size = 32
epochs = 10

# Calculate the number of steps per epoch
train_sequences_count = len(list(SeqIO.parse("Data/Train.fasta", "fasta")))
validation_sequences_count = len(list(SeqIO.parse("Data/Test.fasta", "fasta")))

steps_per_epoch = train_sequences_count // batch_size
validation_steps = validation_sequences_count // batch_size

train_generator = kmer_generator_for_file("Data/Train.fasta",
                                          batch_size=batch_size,
                                          k=3)
validation_generator = kmer_generator_for_file("Data/Train.fasta",
                                               batch_size=batch_size,
                                               k=3)

# Fit the model
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps
)


# Making Predictions

In [ ]:
test_seq, test_labels = kmer_generator_for_file("Data/Test.fasta",
                                                      batch_size=batch_size,
                                                      k=3)



In [ ]:
predictions = model.predict(test_seq)
predicted_labels = (predictions > 0.5).astype(int)


# Evaluation Metrics (Specificity, Sensitivity, AUC, ROC, MCC)

In [ ]:
conf_matrix = confusion_matrix(test_labels, predicted_labels)
tn, fp, fn, tp = conf_matrix.ravel()

# Specificity (True Negative Rate)
specificity = tn / (tn + fp)

# Sensitivity (Recall, True Positive Rate)
sensitivity = tp / (tp + fn)

# AUC and ROC curve
auc = roc_auc_score(true_labels, predictions)
fpr, tpr, thresholds = roc_curve(true_labels, predictions)

# Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(true_labels, predicted_labels)

# Print metrics
print(f'Specificity: {specificity}')
print(f'Sensitivity: {sensitivity}')
print(f'AUC: {auc}')
print(f'MCC: {mcc}')

# Plot ROC Curve

In [ ]:
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (area = {auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()